In [1]:
from utils.dataloader import mel_dataset
from utils.losses import *
from torch.utils.data import DataLoader, random_split

# --- import model ---
from model.Conv2d_model import Conv2d_VAE

# --- import framework ---
import flax 
from flax import jax_utils
import flax.linen as nn
from flax.training import train_state, common_utils
from flax.core.frozen_dict import unfreeze, freeze
import jax
import numpy as np
import jax.numpy as jnp
import optax

from tqdm import tqdm
import os
import wandb
import matplotlib.pyplot as plt
from utils.config_hook import yaml_config_hook

from functools import partial

In [2]:
config_dir = os.path.join(os.path.expanduser('~'),'trainer_module/config')     
config = yaml_config_hook(os.path.join(config_dir, 'config.yaml'))

In [3]:
dataset_dir = '/home/anthonypark6904/dataset'

In [4]:
# --- collate batch for dataloader ---
def collate_batch(batch):
    x_train = [x for x, _ in batch]
    y_train = [y for _, y in batch]                  
        
    return np.array(x_train), np.array(y_train)

In [5]:
def top_n_genre(dataset_dir, n):

    temp_list = []
    dataset_size_list = []
    sort_top_dataset = []

    for k in range(0,30):
        data = mel_dataset(dataset_dir, k)
        temp_list.append(data)
    for i in range(0,30):
        dataset_size_list.append(len(temp_list[i]))
    temp_dict = dict(zip(temp_list, dataset_size_list))

    sort_top_n = sorted(temp_dict.items(), key = lambda item: item[1], reverse=True)[:n]

    for i in range(len(sort_top_n)):
        sort_top_dataset.append(sort_top_n[i][0])

    return sort_top_dataset

In [6]:
dataset_list = top_n_genre(dataset_dir, 5)

Load song_meta.json...


100%|███████████████████████████████| 707989/707989 [00:00<00:00, 783923.10it/s]


Load complete!

Load file list...


165it [00:12, 13.11it/s]


Load song_meta.json...


...

165it [00:12, 13.21it/s]


Load song_meta.json...


100%|███████████████████████████████| 707989/707989 [00:00<00:00, 796016.25it/s]


Load complete!

Load file list...


165it [00:12, 13.26it/s]


In [7]:
from train_module import TrainerModule

2022-09-11 15:23:21.874659: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib


In [8]:
trainer = TrainerModule(seed=125, config=config)

wandb: Currently logged in as: seegong (aiffelthon). Use `wandb login --relogin` to force relogin


In [14]:
dataset_list

In [19]:
dataset_size = len(dataset_list[0])
train_size = int(dataset_size * 0.8)
test_size = dataset_size - train_size

train_dataset, test_dataset = random_split(data, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size']*8, shuffle=True, num_workers=0, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size']*8, shuffle=True, num_workers=0, collate_fn=collate_batch)

genre_encoder=0


In [ ]:
trainer.train_model(train_dataloader, test_dataloader)
trainer.save_model(trainer.state.step)

Epoch 1:   0%|                                           | 0/45 [00:00<?, ?it/s]


ScopeParamShapeError: Inconsistent shapes between value and initializer for parameter "kernel" in "/encoder/Conv_0": (8, 3, 3, 1, 512), (3, 3, 1, 512). (https://flax.readthedocs.io/en/latest/api_reference/flax.errors.html#flax.errors.ScopeParamShapeError)

wandb: Waiting for W&B process to finish... (success).
https://symbolize.stripped_domain/r/?trace=7f52f283ec7f,7f52f279f08f&map= 
*** SIGTERM received by PID 1079676 (TID 1079676) on cpu 57 from PID 1071093; stack trace: ***
PC: @     0x7f52f283ec7f  (unknown)  wait4
    @     0x7f509c2a3294        976  (unknown)
    @     0x7f52f279f090  (unknown)  (unknown)
    @ ... and at least 11 more frames
https://symbolize.stripped_domain/r/?trace=7f52f283ec7f,7f509c2a3293,7f52f279f08f&map=068bf80b76f830987166dd8847d0248f:7f5086d03000-7f509c635de0 
E0911 15:39:57.918036 1079676 coredump_hook.cc:324] RAW: Remote crash gathering disabled for SIGTERM.


In [ ]:
for index, data in enumerate(dataset_list):
    dataset_size = len(data)
    train_size = int(dataset_size * 0.8)
    test_size = dataset_size - train_size

    train_dataset, test_dataset = random_split(data, [train_size, test_size])


    train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size']*8, shuffle=True, num_workers=0, collate_fn=collate_batch)
    test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size']*8, shuffle=True, num_workers=0, collate_fn=collate_batch)
    
    print(f"genre_encoder={index}")
    
    for j in range(len(dataset_list)):
        trainer.train_model(train_dataloader, test_dataloader)
        trainer.save_model(trainer.state.step)

genre_encoder=0


Epoch 1:  24%|████████▎                         | 11/45 [00:25<01:19,  2.34s/it]


ValueError: cannot reshape array of size 7834176 into shape (8,newaxis,48,1876)

NameError: name 'local_device_count' is not defined

SyntaxError: can't use starred expression here (2030247080.py, line 1)